In [397]:
# DSC 540
# Week 3 & 4
# Summer 2024
# Term Project Milestone 2
# Created By Matthew Heinen
# 6/24/24

# Weather and Carlos Santana’s Effectiveness
# DSC 540: Milestone 2 

In [399]:
# Import packages. 

import pandas as pd

In [400]:
# Read in flat file data set from baseball savant. 

# Use read_csv to read the CSV into a DataFrame. 
carlos = pd.read_csv('santana_road_bip.csv')

# Preview first couple lines. 
carlos.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length
0,FF,2013-07-05,101.5,-1.41,6.06,"Rondón, Bruce",467793,541652,single,hit_into_play,...,0,0,7,NaN,NaN,NaN,0.001,0.410,NaN,NaN
1,SI,2018-06-19,101.2,-1.43,6.18,"Hicks, Jordan",467793,663855,field_out,hit_into_play,...,6,6,7,Standard,Standard,245.0,-0.086,-0.232,NaN,NaN
2,FF,2015-05-24,100.9,0.84,6.34,"Chapman, Aroldis",467793,547973,single,hit_into_play,...,5,5,1,Standard,Standard,NaN,0.012,0.578,NaN,NaN
3,FF,2012-09-29,100.9,-1.95,5.90,"Herrera, Kelvin",467793,516969,field_out,hit_into_play,...,6,6,7,NaN,NaN,NaN,-0.201,-0.523,NaN,NaN
4,SI,2019-05-26,100.5,0.86,6.66,"Alvarado, José",467793,621237,force_out,hit_into_play,...,3,3,6,Standard,Standard,143.0,-0.040,-0.459,NaN,NaN


## Transformation 1: Remove Unnecessary Columns

    As we can see above our data set has 94 columns, some of which are either unnecessary or unusable for our project scope. Our first step will be dropping these unnecessary columns. 

First we must print the names of all the columns so we know which ones to drop. We will also print column index in the case we would like to bulk drop. 

In [403]:
# Print column names and their corresponding index

# Loop through each column and print its name and coresponding index. 
for index, column_name in enumerate(carlos.columns):
    print(f"Column: '{column_name}' index: {index}")

Column: 'pitch_type' index: 0
Column: 'game_date' index: 1
Column: 'release_speed' index: 2
Column: 'release_pos_x' index: 3
Column: 'release_pos_z' index: 4
Column: 'player_name' index: 5
Column: 'batter' index: 6
Column: 'pitcher' index: 7
Column: 'events' index: 8
Column: 'description' index: 9
Column: 'spin_dir' index: 10
Column: 'spin_rate_deprecated' index: 11
Column: 'break_angle_deprecated' index: 12
Column: 'break_length_deprecated' index: 13
Column: 'zone' index: 14
Column: 'des' index: 15
Column: 'game_type' index: 16
Column: 'stand' index: 17
Column: 'p_throws' index: 18
Column: 'home_team' index: 19
Column: 'away_team' index: 20
Column: 'type' index: 21
Column: 'hit_location' index: 22
Column: 'bb_type' index: 23
Column: 'balls' index: 24
Column: 'strikes' index: 25
Column: 'game_year' index: 26
Column: 'pfx_x' index: 27
Column: 'pfx_z' index: 28
Column: 'plate_x' index: 29
Column: 'plate_z' index: 30
Column: 'on_3b' index: 31
Column: 'on_2b' index: 32
Column: 'on_1b' inde

In [404]:
# Remove all fielder values 
# Use drop method remove columns 59-67
carlos = carlos.drop(carlos.columns[59:68], axis=1)

# View new size of df to validate drop. 
carlos.shape

(2831, 85)

In [405]:
# Remove batter index as all batters are Carlos Santana. 
carlos = carlos.drop('batter', axis=1)

# Remove description as all are balls hit into play 
carlos = carlos.drop('description', axis=1)

# Remove game year as its rebetitive with date. 
carlos = carlos.drop('game_year', axis=1)

# View new size of df to validate drop. 
carlos.shape

(2831, 82)

## Transformation 2: Make on_1b , on_2b and on_3b Binary. 

Here we are not concerned with which player is on base just which bases are occupied. This will still allow us to evaluate situational hitting and clean our database making it easier to analyze later. 

In [407]:
# Build function to test if a column had content and apply a binary based on the result. 

def replace_with_binary(value):
    #If the value is empty return zero. 
    if pd.isna(value):
        return 0
    # Add clause in case you run code twice. 
    elif value == 0:
        return 0 
    #If there is a runner on the base return 1. 
    else:
        return 1

In [408]:
# Use the apply method to loop through each of the on base columns and call 
# replace_with_binary function. 

# First base. 
carlos['on_1b'] = carlos['on_1b'].apply(replace_with_binary)

# Second base. 
carlos['on_2b'] = carlos['on_2b'].apply(replace_with_binary)

# Third base. 
carlos['on_3b'] = carlos['on_3b'].apply(replace_with_binary)

# Preview the results of the onbase columns. 
print(carlos[['on_1b', 'on_2b', 'on_3b']].head(10))


   on_1b  on_2b  on_3b
0      1      0      0
1      0      0      0
2      0      1      0
3      1      1      1
4      1      0      1
5      0      1      0
6      0      0      0
7      0      0      0
8      1      1      0
9      1      0      0


# Transformation 3: convert date into Unix time (UTC time zone). 

This transformation will be done by assigning a start time of 7 pm (most common start time in baseball to all games. Additionally a transformation will be done to unix time which is required for our API weather call. 

In [410]:
# Convert date column to Unix time
# Add this column to the Data Frame
# Use game_date and to_datetime method to convert to unix. 
carlos['unix_time'] = pd.to_datetime(carlos['game_date']).astype(int) // 10**9

# Add a column that assumes a 7pm start time for every game. 
# Convert Unix time to 7pm
# During daylight savings, CDT is 5 hours behing UTC. 
# to get 7pm or 19:00 we need to add 5 more hours to catch up making for 24 hours. 
carlos['unix_time'] = carlos['unix_time'] + (24 * 3600)  # 26 hours * 3600 seconds/hour

# Create a game date_time to validate the Unix time. 


# Preview the result of the time related columns. 
# Preview the results of the onbase columns. 
print(carlos[['game_date', 'unix_time']].head(10))

    game_date   unix_time
0  2013-07-05  1373068800
1  2018-06-19  1529452800
2  2015-05-24  1432512000
3  2012-09-29  1348963200
4  2019-05-26  1558915200
5  2023-06-03  1685836800
6  2019-09-14  1568505600
7  2022-04-11  1649721600
8  2018-06-25  1529971200
9  2012-07-08  1341792000


Here we see that we have left our game date and converted to 7pm Central in UTC Unix time 